# Test Cubeviz Base Functionality with Glupyter

This work is to satisfy https://github.com/spacetelescope/dat_pyinthesky/issues/28 . Example adapted from https://mybinder.org/v2/gh/glue-viz/glue-jupyter/master?filepath=notebooks%2FAstronomy%2FL1448%2FL1448%20in%203D.ipynb .

1. Load a data cube
2. Display 2D and 1D
3. Median collapse cube
4. Stretch goal: Hover stats with X, Y, value, RA, DEC

In [ ]:
import numpy as np

import glue_jupyter as gj
from glue_jupyter.data import require_data

Load a sample cube.

In [ ]:
# Sample data magic.
require_data('Astronomy/L1448/l1448_13co.fits')

In [ ]:
# Start the application.
app = gj.jglue()

In [ ]:
cube = app.load_data('l1448_13co.fits')
print(cube.components)

Extract data array.

In [ ]:
data_array = cube['PRIMARY']
print(data_array.shape)

**Hacky hack hack: Bloat the array up to MIRI size.**

This hack also renders the RA, DEC, and wavelength space useless.

In [ ]:
data_array = np.tile(data_array, (19, 1, 1))
print(data_array.shape)

In [ ]:
from glue.core import Data

big_cube = Data(PRIMARY=data_array)
print(big_cube.components)
print(big_cube.shape)

In [ ]:
big_cube = app.add_data(big_cube=big_cube)[0]

Compute its median-collapsed cube.

In [ ]:
median_collapsed_cube = np.nanmedian(data_array, axis=0)

In [ ]:
mcc_im = app.add_data(mcc=median_collapsed_cube)[0]

Display 3D.

**TODO: Z display looks weird, does not show all data?**

**TODO: Has 2 FutureWarning from ipyvolumes!**

In [ ]:
app.volshow(data=big_cube)

Display 2D. This needs https://github.com/glue-viz/glue-jupyter/pull/74 to work.

**TODO: This does not show the data, I only see empty viewer. A bug?**

**TODO: How do select which 2D slice to display, by sliding the World0 slider?**

In [ ]:
app.imshow(data=big_cube)

Display 1D.

**TODO: How do select location on X and Y to probe through Z?**

**TODO: Showed something for cube but nothing for big_cube.**

In [ ]:
app.profile1d('Pixel Axis 0 [z]', data=big_cube)

Display median-collapsed cube.

**TODO: This does not show the data, I only see empty viewer. A bug?**

In [ ]:
app.imshow(data=mcc_im)

Hover stats (stretch goal).

**TODO: I am not even going to try this one. Tom R, halp!!!**